# GTEx model with different priors

## Load libraries

In [1]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

# 3. Install PLIER2 (mchikina/PLIER2) if not already installed
if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2"  # adjust
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER2)
library(clusterProfiler)
library(org.Hs.eg.db)
library(httr)
library(tidyr)
library(AnnotationDbi)

source(here("config.R"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses



clusterProfiler v4.10.0  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGener

## Output directory

In [2]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

In [3]:
gtex_plier2_kegg <- readRDS(file.path(output_data_dir, "gtex_KEGG_2021_Human_PLIER2.rds"))

kegg_plier2_summary <- gtex_plier2_kegg$summary %>%
    dplyr::filter(FDR < 0.05 & AUC > 0.7) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway = paste(pathway, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

In [4]:
gtex_baseRes <- readRDS(file.path(output_data_dir, "gtex_PLIER2_baseRes.rds"))

In [5]:
gtex_genes <- readRDS(file.path(output_data_dir, "gtex_genes.rds"))

In [6]:
colnames(gtex_baseRes$Z) <- paste0('LV', seq_len(ncol(gtex_baseRes$Z)))
rownames(gtex_baseRes$Z) <- gtex_genes

In [7]:
head(gtex_baseRes$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.0000000,0,0,0.1239626,0.0000000,0.0000000
RP11-34P13.15,0,0,0,0,0,0.3678263,0,0,0,0,⋯,0,0,0,0,0.3107976,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.16,0,0,0,0,0,0.3559246,0,0,0,0,⋯,0,0,0,0,0.3337613,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.18,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.2021179,0,0,0.2418167,0.0000000,0.0000000
AP006222.2,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.3997058,0,0,0.0000000,0.1570023,0.0000000
MTND1P23,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.4758745,0,0,0.0000000,0.0000000,0.4125261


In [8]:
kegg_plier2_Z <- data.frame(gtex_baseRes$Z)

In [9]:
Z <- gtex_baseRes$Z

map <- bitr(rownames(Z), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)
map <- map[!duplicated(map$SYMBOL), ]

lv_names <- colnames(Z)

top_syms <- setNames(vector("list", length(lv_names)), lv_names)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(rownames(Z), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db):
“20.4% of input gene IDs are fail to map...”


In [10]:
head(top_syms$LV1)

NULL

In [11]:
kegg_plier2_Z %>% 
dplyr::select(LV1)  %>% 
arrange(desc(LV1)) %>%
tail(10)

,LV1
,<dbl>
MT-ND4,0
MT-TH,0
MT-TS2,0
MT-TL2,0
MT-ND5,0
MT-ND6,0
MT-TE,0
MT-CYB,0
MT-TT,0


In [12]:
for (lv in lv_names) {
  v <- Z[, lv]
  idx <- which(!is.na(v) & v > 0)
  if (length(idx) == 0L) { top_syms[[lv]] <- character(0); next }
  ord <- idx[order(v[idx], decreasing = TRUE)]
  top_syms[[lv]] <- rownames(Z)[ord]
}

top_entrez <- lapply(top_syms, function(syms) {
  ids <- map$ENTREZID[match(syms, map$SYMBOL)]
  unique(as.character(ids[!is.na(ids)]))
})

kegg_list <- lapply(top_entrez, function(ids) {
  if (!length(ids)) return(NULL)
  enrichKEGG(gene = ids, organism = "hsa", qvalueCutoff = 0.05)
})

Reading KEGG annotation online: "https://rest.kegg.jp/link/hsa/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/hsa"...



In [13]:
kegg_list$LV15@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Peroxisome"                                       
 [2] "Fatty acid degradation"                           
 [3] "Valine, leucine and isoleucine degradation"       
 [4] "Carbon metabolism"                                
 [5] "Biosynthesis of cofactors"                        
 [6] "Glycine, serine and threonine metabolism"         
 [7] "Fatty acid metabolism"                            
 [8] "Tryptophan metabolism"                            
 [9] "beta-Alanine metabolism"                          
[10] "PPAR signaling pathway"                           
[11] "Complement and coagulation cascades"              
[12] "One carbon pool by folate"                        
[13] "Alcoholic liver disease"                          
[14] "Pyruvate metabolism"                              
[15] "Propanoate metabolism"                            
[16] "Drug metabolism - cytochrome P450"                
[17] "Tyrosine metabolism"                              
[18] "Metabolism of xenobiotics by cytochrome P450"     
[19] "Butanoate metabolism"                             
[20] "Histidine metabolism"                             
[21] "Glyoxylate and dicarboxylate metabolism"          
[22] "Retinol metabolism"                               
[23] "Biosynthesis of unsaturated fatty acids"          
[24] "Arginine and proline metabolism"                  
[25] "Folate biosynthesis"                              
[26] "Arginine biosynthesis"                            
[27] "Alanine, aspartate and glutamate metabolism"      
[28] "Biosynthesis of amino acids"                      
[29] "Nicotinate and nicotinamide metabolism"           
[30] "Folate transport and metabolism"                  
[31] "Pantothenate and CoA biosynthesis"                
[32] "Cysteine and methionine metabolism"               
[33] "Chemical carcinogenesis - DNA adducts"            
[34] "Glycolysis / Gluconeogenesis"                     
[35] "Vitamin digestion and absorption"                 
[36] "Cholesterol metabolism"                           
[37] "Selenocompound metabolism"                        
[38] "Drug metabolism - other enzymes"                  
[39] "2-Oxocarboxylic acid metabolism"                  
[40] "Glutathione metabolism"                           
[41] "Ferroptosis"                                      
[42] "ABC transporters"                                 
[43] "Glycerolipid metabolism"                          
[44] "Citrate cycle (TCA cycle)"                        
[45] "Lysine degradation"                               
[46] "Pentose phosphate pathway"                        
[47] "Terpenoid backbone biosynthesis"                  
[48] "Fatty acid elongation"                            
[49] "Steroid biosynthesis"                             
[50] "Lipoic acid metabolism"                           
[51] "Biosynthesis of nucleotide sugars"                
[52] "Sulfur metabolism"                                
[53] "Primary bile acid biosynthesis"                   
[54] "Glycerophospholipid metabolism"                   
[55] "Insulin resistance"                               
[56] "AMPK signaling pathway"                           
[57] "Pyrimidine metabolism"                            
[58] "Fat digestion and absorption"                     
[59] "Antifolate resistance"                            
[60] "Mineral absorption"                               
[61] "Ascorbate and aldarate metabolism"                
[62] "Nucleotide metabolism"                            
[63] "Steroid hormone biosynthesis"                     
[64] "Chemical carcinogenesis - reactive oxygen species"
[65] "Proximal tubule bicarbonate reclamation"          
[66] "Adipocytokine signaling pathway"                  
[67] "Amino sugar and nucleotide sugar metabolism"      
[68] "Glucagon signaling pathway"

In [14]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV15")

LV,pathway
<chr>,<chr>
LV15,"Drug metabolism, Glycine, serine and threonine metabolism, Peroxisome, Retinol metabolism, Tyrosine metabolism"


In [15]:
kegg_list$LV102@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Osteoclast differentiation"                                   
  [2] "Kaposi sarcoma-associated herpesvirus infection"              
  [3] "TNF signaling pathway"                                        
  [4] "Epstein-Barr virus infection"                                 
  [5] "Leishmaniasis"                                                
  [6] "Herpes simplex virus 1 infection"                             
  [7] "Malaria"                                                      
  [8] "Lipid and atherosclerosis"                                    
  [9] "Cytokine-cytokine receptor interaction"                       
 [10] "Pertussis"                                                    
 [11] "Alcoholic liver disease"                                      
 [12] "Toll-like receptor signaling pathway"                         
 [13] "Complement and coagulation cascades"                          
 [14] "Legionellosis"                                                
 [15] "Tuberculosis"                                                 
 [16] "NF-kappa B signaling pathway"                                 
 [17] "Phagosome"                                                    
 [18] "Toxoplasmosis"                                                
 [19] "Pathogenic Escherichia coli infection"                        
 [20] "MAPK signaling pathway"                                       
 [21] "AGE-RAGE signaling pathway in diabetic complications"         
 [22] "NOD-like receptor signaling pathway"                          
 [23] "Chagas disease"                                               
 [24] "Influenza A"                                                  
 [25] "Human T-cell leukemia virus 1 infection"                      
 [26] "Rheumatoid arthritis"                                         
 [27] "Efferocytosis"                                                
 [28] "Th17 cell differentiation"                                    
 [29] "Apoptosis"                                                    
 [30] "Inflammatory bowel disease"                                   
 [31] "Human cytomegalovirus infection"                              
 [32] "Measles"                                                      
 [33] "Salmonella infection"                                         
 [34] "HIF-1 signaling pathway"                                      
 [35] "Coronavirus disease - COVID-19"                               
 [36] "IL-17 signaling pathway"                                      
 [37] "Cellular senescence"                                          
 [38] "JAK-STAT signaling pathway"                                   
 [39] "Staphylococcus aureus infection"                              
 [40] "Hepatitis B"                                                  
 [41] "PI3K-Akt signaling pathway"                                   
 [42] "Human immunodeficiency virus 1 infection"                     
 [43] "Cell adhesion molecules"                                      
 [44] "p53 signaling pathway"                                        
 [45] "African trypanosomiasis"                                      
 [46] "Fluid shear stress and atherosclerosis"                       
 [47] "Pancreatic cancer"                                            
 [48] "Amoebiasis"                                                   
 [49] "C-type lectin receptor signaling pathway"                     
 [50] "Hematopoietic cell lineage"                                   
 [51] "Natural killer cell mediated cytotoxicity"                    
 [52] "Transcriptional misregulation in cancer"                      
 [53] "Intestinal immune network for IgA production"                 
 [54] "Colorectal cancer"                                            
 [55] "Allograft rejection"                                          
 [56] "Acute myeloid leukemia"                                       
 [57] "Chronic myeloid leukemia"                                     
 [58] "Fc ga

In [16]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV102")

LV,pathway
<chr>,<chr>
LV102,"Complement and coagulation cascades, Systemic lupus erythematosus"


In [17]:
kegg_list$LV12@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Salmonella infection"                                         
  [2] "Endocytosis"                                                  
  [3] "Shigellosis"                                                  
  [4] "Osteoclast differentiation"                                   
  [5] "Epstein-Barr virus infection"                                 
  [6] "Yersinia infection"                                           
  [7] "Tuberculosis"                                                 
  [8] "NOD-like receptor signaling pathway"                          
  [9] "Phagosome"                                                    
 [10] "Lysosome"                                                     
 [11] "Pathogenic Escherichia coli infection"                        
 [12] "Human immunodeficiency virus 1 infection"                     
 [13] "Kaposi sarcoma-associated herpesvirus infection"              
 [14] "Herpes simplex virus 1 infection"                             
 [15] "Human cytomegalovirus infection"                              
 [16] "Influenza A"                                                  
 [17] "Natural killer cell mediated cytotoxicity"                    
 [18] "PD-L1 expression and PD-1 checkpoint pathway in cancer"       
 [19] "Apoptosis"                                                    
 [20] "Fc gamma R-mediated phagocytosis"                             
 [21] "Chemokine signaling pathway"                                  
 [22] "Viral carcinogenesis"                                         
 [23] "B cell receptor signaling pathway"                            
 [24] "Lipid and atherosclerosis"                                    
 [25] "Autophagy - animal"                                           
 [26] "Measles"                                                      
 [27] "Th17 cell differentiation"                                    
 [28] "Hepatitis B"                                                  
 [29] "Proteasome"                                                   
 [30] "Acute myeloid leukemia"                                       
 [31] "Chagas disease"                                               
 [32] "NF-kappa B signaling pathway"                                 
 [33] "Toll-like receptor signaling pathway"                         
 [34] "C-type lectin receptor signaling pathway"                     
 [35] "Neurotrophin signaling pathway"                               
 [36] "Platelet activation"                                          
 [37] "Hepatitis C"                                                  
 [38] "Leishmaniasis"                                                
 [39] "SNARE interactions in vesicular transport"                    
 [40] "Necroptosis"                                                  
 [41] "Pancreatic cancer"                                            
 [42] "Legionellosis"                                                
 [43] "Neutrophil extracellular trap formation"                      
 [44] "Sphingolipid signaling pathway"                               
 [45] "T cell receptor signaling pathway"                            
 [46] "Non-small cell lung cancer"                                   
 [47] "Chronic myeloid leukemia"                                     
 [48] "Regulation of actin cytoskeleton"                             
 [49] "Leukocyte transendothelial migration"                         
 [50] "Human papillomavirus infection"                               
 [51] "Pathways of neurodegeneration - multiple diseases"            
 [52] "Pertussis"                                                    
 [53] "TNF signaling pathway"                                        
 [54] "Fc epsilon RI signaling pathway"                              
 [55] "Human T-cell leukemia virus 1 infection"                      
 [56] "Th1 and Th2 cell differentiation"                             
 [57] "Vibrio cholerae infection"                                    
 [58] "HIF-1

In [18]:
gtex_plier2_kegg$summary %>%
dplyr::filter(LV == "LV23") %>% 
dplyr::filter(FDR < 0.05)

pathway,LV,AUC,p-value,FDR
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Coronavirus disease,LV23,0.6814745,9.686035e-06,1.456324e-04
Legionellosis,LV23,0.7086666,4.968835e-03,1.953899e-02
Pancreatic secretion,LV23,0.7074812,7.211141e-04,4.151278e-03
RNA transport,LV23,0.6892528,4.377337e-05,4.975409e-04
Ribosome,LV23,0.8209393,1.150944e-09,1.337187e-07
Spliceosome,LV23,0.6393640,4.072564e-03,1.678947e-02


In [19]:
kegg_list$LV23@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

kegg_plier2_summary  %>%  dplyr::filter(LV == "LV23")

[1] "Coronavirus disease - COVID-19"                   
 [2] "Pancreatic secretion"                             
 [3] "Protein processing in endoplasmic reticulum"      
 [4] "DNA replication"                                  
 [5] "Non-alcoholic fatty liver disease"                
 [6] "Nucleotide metabolism"                            
 [7] "Mitophagy - animal"                               
 [8] "Base excision repair"                             
 [9] "Protein digestion and absorption"                 
[10] "Diabetic cardiomyopathy"                          
[11] "Protein export"                                   
[12] "Chemical carcinogenesis - reactive oxygen species"

LV,pathway
<chr>,<chr>
LV23,"Legionellosis, Pancreatic secretion, Ribosome"


In [20]:
kegg_list$LV44@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

kegg_plier2_summary  %>%  dplyr::filter(LV == "LV44")

[1] "Fatty acid metabolism"                                   
 [2] "Carbon metabolism"                                       
 [3] "Fatty acid degradation"                                  
 [4] "PPAR signaling pathway"                                  
 [5] "Glycine, serine and threonine metabolism"                
 [6] "Biosynthesis of unsaturated fatty acids"                 
 [7] "Propanoate metabolism"                                   
 [8] "Neuroactive ligand signaling"                            
 [9] "beta-Alanine metabolism"                                 
[10] "Calcium signaling pathway"                               
[11] "Arginine and proline metabolism"                         
[12] "Biosynthesis of amino acids"                             
[13] "Thyroid hormone signaling pathway"                       
[14] "Valine, leucine and isoleucine degradation"              
[15] "Fatty acid elongation"                                   
[16] "Peroxisome"                                              
[17] "Alanine, aspartate and glutamate metabolism"             
[18] "GABAergic synapse"                                       
[19] "Hippo signaling pathway"                                 
[20] "One carbon pool by folate"                               
[21] "Glutamatergic synapse"                                   
[22] "Proteoglycans in cancer"                                 
[23] "Efferocytosis"                                           
[24] "Axon guidance"                                           
[25] "Dopaminergic synapse"                                    
[26] "Notch signaling pathway"                                 
[27] "Basal cell carcinoma"                                    
[28] "Wnt signaling pathway"                                   
[29] "Central carbon metabolism in cancer"                     
[30] "Hedgehog signaling pathway"                              
[31] "Hepatocellular carcinoma"                                
[32] "Lysine degradation"                                      
[33] "Morphine addiction"                                      
[34] "Nitrogen metabolism"                                     
[35] "Alcoholic liver disease"                                 
[36] "Circadian entrainment"                                   
[37] "Mineral absorption"                                      
[38] "Cysteine and methionine metabolism"                      
[39] "Pyruvate metabolism"                                     
[40] "Ferroptosis"                                             
[41] "Purine metabolism"                                       
[42] "TGF-beta signaling pathway"                              
[43] "Choline metabolism in cancer"                            
[44] "Rap1 signaling pathway"                                  
[45] "Human papillomavirus infection"                          
[46] "Regulation of actin cytoskeleton"                        
[47] "Long-term depression"                                    
[48] "Glycolysis / Gluconeogenesis"                            
[49] "Long-term potentiation"                                  
[50] "Nicotine addiction"                                      
[51] "Sphingolipid signaling pathway"                          
[52] "Pentose phosphate pathway"                               
[53] "Gap junction"                                            
[54] "cAMP signaling pathway"                                  
[55] "Cytoskeleton in muscle cells"                            
[56] "Breast cancer"                                           
[57] "Tryptophan metabolism"                                   
[58] "Butanoate metabolism"                                    
[59] "Signaling pathways regulating pluripotency of stem cells"
[60] "Focal adhesion"                                          
[61] "Proximal tubule bicarbonate reclamation"                 
[62] "HIF-1 signaling pathway"                                 
[63] "Aldosterone synthesis and s

LV,pathway
<chr>,<chr>
LV44,"Alanine, aspartate and glutamate metabolism, Alcoholism, GABAergic synapse"


# GTEx tissues

## ORA

In [21]:
# Download & parse Enrichr GTEx tissues 
enrichr_url <- "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023"
txt <- content(GET(enrichr_url), as = "text", encoding = "UTF-8")
lines <- strsplit(txt, "\n", fixed = TRUE)[[1]]
parts <- strsplit(lines, "\t", fixed = TRUE)

terms <- vapply(parts, function(x) if (length(x)) x[[1]] else NA_character_, character(1))
genes_list <- lapply(parts, function(x) {
  if (length(x) <= 2) x[2:length(x)] else x[3:length(x)]  
})

GTEx_TERM2GENE_SYM <- tibble(
  term = rep(terms, lengths(genes_list)),
  SYMBOL = unlist(genes_list, use.names = FALSE)
) %>%
  filter(!is.na(term), !is.na(SYMBOL), SYMBOL != "")

sym2eg <- AnnotationDbi::select(
  org.Hs.eg.db,
  keys = unique(GTEx_TERM2GENE_SYM$SYMBOL),
  columns = "ENTREZID",
  keytype = "SYMBOL"
) %>% distinct(SYMBOL, ENTREZID) %>% filter(!is.na(ENTREZID))

GTEx_TERM2GENE <- GTEx_TERM2GENE_SYM %>%
  inner_join(sym2eg, by = "SYMBOL") %>%
  transmute(term, gene = ENTREZID)


# Run ORA for each LV gene set against GTEx 
gtex_list <- lapply(top_entrez, function(ids) {
  if (!length(ids)) return(NULL)
  enricher(
    gene           = ids,
    TERM2GENE      = GTEx_TERM2GENE,
    pvalueCutoff   = 0.05,
    pAdjustMethod  = "BH",
  )
})

'select()' returned 1:many mapping between keys and columns

Warning message in inner_join(., sym2eg, by = "SYMBOL"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 4590 of `x` matches multiple rows in `y`.
ℹ Row 253 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [22]:
gtex_meta <- read.table(
    here('data/gtex/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt'), 
    sep = '\t', 
    header = TRUE,
    quote = "", 
    fill = TRUE,
    comment.char = "" 
)

Let's use GTEx metadata to see which approach (PLIERfull or PLIERbase + ORA) can better capture the tissue-specific

In [23]:
head(gtex_meta)

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<int>,<dbl>
1,GTEX-1117F-0003-SM-58Q7G,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,GTEX-1117F-0003-SM-5DWSB,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,GTEX-1117F-0003-SM-6WBT7,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,GTEX-1117F-0011-R10a-SM-AHZ7F,NA,"B1, A1",,NA,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,GTEX-1117F-0011-R10b-SM-CYKQ8,NA,"B1, A1",,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,⋯,14648800,11999300,0.00315785,14669500,50.0354,0.00310538,0.99474,NA,0,50.1944


In [24]:
gtex_meta_sub <- gtex_meta  %>% 
dplyr::select(SAMPID, SMTS, SMTSD) %>% 
    dplyr::rename(sample =SAMPID, tissue = SMTS, tissue_detail = SMTSD)

In [25]:
gtex_samples <- readRDS(here("output/gtex/gtex_samples.rds"))

In [26]:
gtex_baseRes_B  <- data.frame(gtex_baseRes$B)

In [27]:
colnames(gtex_baseRes_B) <- gtex_samples

In [28]:
rownames(gtex_baseRes_B) <- paste0('LV', seq_len(nrow(gtex_baseRes_B)))

In [29]:
lv1_df <- gtex_baseRes_B %>%
  tibble::rownames_to_column("LV") %>%
  filter(LV == "LV1") %>%
  tidyr::pivot_longer(-LV, names_to = "sample", values_to = "score") %>%
  arrange(desc(score))

In [30]:
lv1_df %>% 
dplyr::left_join(gtex_meta_sub)  %>% 
head(50)

Joining with `by = join_by(sample)`


LV,sample,score,tissue,tissue_detail
<chr>,<chr>,<dbl>,<chr>,<chr>
LV1,GTEX-17EUY-0326-SM-7DUEY,0.8289919,Blood Vessel,Artery - Tibial
LV1,GTEX-SN8G-0526-SM-32PLE,0.8192388,Blood Vessel,Artery - Tibial
LV1,GTEX-1CB4F-0726-SM-793AO,0.8126255,Blood Vessel,Artery - Tibial
LV1,GTEX-POMQ-2126-SM-2S1OJ,0.7618157,Blood Vessel,Artery - Tibial
LV1,GTEX-1GF9W-0226-SM-7PC2U,0.7553025,Blood Vessel,Artery - Tibial
LV1,GTEX-15UF6-2026-SM-6LPI3,0.7515070,Ovary,Ovary
LV1,GTEX-14AS3-2326-SM-5S2NZ,0.7498816,Blood Vessel,Artery - Tibial
LV1,GTEX-R53T-2026-SM-3GIJF,0.7416174,Blood Vessel,Artery - Tibial
LV1,GTEX-P44H-0626-SM-2XCF2,0.7360011,Blood Vessel,Artery - Tibial


In [31]:
gtex_list$LV1@result %>%
  filter(p.adjust < 0.05) %>%
  pull(Description) %>%
  unique()

[1] "Artery - Tibial Male 30-39 Up"                        
 [2] "Artery - Tibial Female 40-49 Up"                      
 [3] "Artery - Tibial Female 60-69 Up"                      
 [4] "Artery - Tibial Female 50-59 Up"                      
 [5] "Artery - Tibial Female 20-29 Up"                      
 [6] "Artery - Tibial Female 30-39 Up"                      
 [7] "Artery - Tibial Male 20-29 Up"                        
 [8] "Artery - Tibial Male 40-49 Up"                        
 [9] "Artery - Tibial Male 50-59 Up"                        
[10] "Artery - Tibial Female 70-79 Up"                      
[11] "Artery - Tibial Male 60-69 Up"                        
[12] "Colon - Sigmoid Female 70-79 Up"                      
[13] "Artery - Aorta Male 70-79 Up"                         
[14] "Colon - Sigmoid Male 30-39 Up"                        
[15] "Artery - Aorta Female 60-69 Up"                       
[16] "Colon - Sigmoid Male 20-29 Up"                        
[17] "Artery - Tibial Male 70-79 Up"                        
[18] "Artery - Aorta Male 50-59 Up"                         
[19] "Colon - Sigmoid Female 40-49 Up"                      
[20] "Artery - Aorta Female 30-39 Up"                       
[21] "Artery - Aorta Female 40-49 Up"                       
[22] "Colon - Sigmoid Male 50-59 Up"                        
[23] "Colon - Sigmoid Female 50-59 Up"                      
[24] "Artery - Coronary Male 70-79 Up"                      
[25] "Colon - Sigmoid Female 20-29 Up"                      
[26] "Colon - Sigmoid Male 40-49 Up"                        
[27] "Ovary Female 70-79 Up"                                
[28] "Artery - Aorta Female 50-59 Up"                       
[29] "Colon - Sigmoid Female 30-39 Up"                      
[30] "Colon - Sigmoid Female 60-69 Up"                      
[31] "Artery - Aorta Male 40-49 Up"                         
[32] "Artery - Aorta Male 30-39 Up"                         
[33] "Esophagus - Gastroesophageal Junction Female 20-29 Up"
[34] "Esophagus - Gastroesophageal Junction Male 30-39 Up"  
[35] "Esophagus - Gastroesophageal Junction Female 50-59 Up"
[36] "Artery - Aorta Male 60-69 Up"                         
[37] "Artery - Aorta Female 70-79 Up"                       
[38] "Artery - Aorta Male 20-29 Up"                         
[39] "Esophagus - Gastroesophageal Junction Female 30-39 Up"
[40] "Esophagus - Muscularis Female 60-69 Up"               
[41] "Ovary Female 60-69 Up"                                
[42] "Artery - Aorta Female 20-29 Up"                       
[43] "Esophagus - Muscularis Female 70-79 Up"               
[44] "Esophagus - Muscularis Male 20-29 Up"                 
[45] "Esophagus - Gastroesophageal Junction Male 50-59 Up"  
[46] "Esophagus - Muscularis Male 50-59 Up"                 
[47] "Ovary Female 30-39 Up"                                
[48] "Esophagus - Gastroesophageal Junction Male 60-69 Up"  
[49] "Esophagus - Muscularis Female 40-49 Up"               
[50] "Esophagus - Gastroesophageal Junction Male 20-29 Up"  
[51] "Esophagus - Muscularis Female 20-29 Up"               
[52] "Esophagus - Muscularis Female 30-39 Up"               
[53] "Esophagus - Muscularis Male 30-39 Up"                 
[54] "Esophagus - Gastroesophageal Junction Female 40-49 Up"
[55] "Esophagus - Muscularis Male 40-49 Up"

In [32]:
gtex_plier2_gtex <- readRDS(file.path(output_data_dir, "gtex_GTEx_Tissues_V8_2023_PLIER2.rds"))

In [33]:
gtex_plier2_gtex_summary <- gtex_plier2_gtex$summary %>%
    dplyr::filter(FDR < 0.05 & AUC > 0.7) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway = paste(pathway, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

In [34]:
gtex_plier2_gtex_summary  %>% 
dplyr::filter(LV == "LV1")  %>% 
dplyr::pull(pathway)  %>%
  unique()

[1] "Artery - Tibial Male 60-69 Up, Artery - Aorta Female 60-69 Up, Artery - Aorta Female 20-29 Up, Artery - Coronary Male 30-39 Up, Artery - Aorta Female 40-49 Up, Artery - Aorta Female 70-79 Up"

In [54]:
lv15_df <- gtex_baseRes_B %>%
    tibble::rownames_to_column("LV") %>%
    filter(LV == "LV15") %>%
    tidyr::pivot_longer(-LV, names_to = "sample", values_to = "score") %>%
    arrange(desc(score)) %>%
    dplyr::left_join(gtex_meta_sub)

top_n <- ceiling(0.05 * nrow(lv15_df))
lv15_df %>% slice_head(n = top_n) %>% 
pull(tissue) %>%
table()

lv15_df %>% 
    slice_head(n = top_n) %>% 
    count(tissue) %>% 
    mutate(percent = 100 * n / sum(n))

lv15_df %>% 
    slice_head(n = top_n) %>% 
    count(tissue) %>% 
    arrange(desc(n)) %>% 
    mutate(percent = 100 * as.numeric(n) / sum(as.numeric(n))) %>% 
    mutate(msg = paste0(tissue, ": ", round(percent, 1), "%")) %>% 
    { paste(.$msg, collapse = "; ") }

Joining with `by = join_by(sample)`


.
 Adipose Tissue   Adrenal Gland           Blood    Blood Vessel          Breast 
             23             137              12               5              17 
          Colon       Esophagus           Heart          Kidney           Liver 
             51               1               1              66             226 
          Ovary        Pancreas        Prostate            Skin Small Intestine 
             27              69              20               3              67 
         Spleen         Stomach         Thyroid          Uterus 
              1             133              10               1 

tissue,n,percent
<chr>,<int>,<dbl>
Adipose Tissue,23,2.6436782
Adrenal Gland,137,15.7471264
Blood,12,1.3793103
Blood Vessel,5,0.5747126
Breast,17,1.9540230
Colon,51,5.8620690
Esophagus,1,0.1149425
Heart,1,0.1149425
Kidney,66,7.5862069


[1] "Liver: 26%; Adrenal Gland: 15.7%; Stomach: 15.3%; Pancreas: 7.9%; Small Intestine: 7.7%; Kidney: 7.6%; Colon: 5.9%; Ovary: 3.1%; Adipose Tissue: 2.6%; Prostate: 2.3%; Breast: 2%; Blood: 1.4%; Thyroid: 1.1%; Blood Vessel: 0.6%; Skin: 0.3%; Esophagus: 0.1%; Heart: 0.1%; Spleen: 0.1%; Uterus: 0.1%"

In [53]:
gtex_list$LV15@result %>%
  filter(p.adjust < 0.05) %>%
  pull(Description) %>%
  unique()


tissues <- gtex_list$LV15@result %>%
  dplyr::filter(p.adjust < 0.05) %>%
  dplyr::pull(Description) %>%
  stringr::str_replace("\\s+-\\s+.*$", "") %>%          
  stringr::str_replace("\\s+(Male|Female)\\b.*$", "") %>% 
  stringr::str_replace("\\s+Up$", "") %>%                
  stringr::str_squish() %>%
  unique() %>%
  sort()

paste(tissues, collapse = ", ")

[1] "Liver Male 20-29 Up"                         
 [2] "Liver Female 60-69 Up"                       
 [3] "Liver Female 20-29 Up"                       
 [4] "Liver Male 50-59 Up"                         
 [5] "Liver Male 40-49 Up"                         
 [6] "Liver Male 70-79 Up"                         
 [7] "Liver Female 40-49 Up"                       
 [8] "Liver Female 50-59 Up"                       
 [9] "Liver Male 30-39 Up"                         
[10] "Liver Male 60-69 Up"                         
[11] "Liver Female 30-39 Up"                       
[12] "Adrenal Gland Male 70-79 Up"                 
[13] "Adrenal Gland Female 50-59 Up"               
[14] "Adrenal Gland Female 30-39 Up"               
[15] "Adrenal Gland Male 60-69 Up"                 
[16] "Adrenal Gland Female 60-69 Up"               
[17] "Adrenal Gland Male 50-59 Up"                 
[18] "Adrenal Gland Male 40-49 Up"                 
[19] "Adrenal Gland Female 40-49 Up"               
[20] "Adrenal Gland Female 20-29 Up"               
[21] "Adrenal Gland Male 20-29 Up"                 
[22] "Adipose - Subcutaneous Female 20-29 Up"      
[23] "Adrenal Gland Male 30-39 Up"                 
[24] "Adipose - Subcutaneous Male 20-29 Up"        
[25] "Adipose - Visceral (Omentum) Female 20-29 Up"
[26] "Adipose - Subcutaneous Female 30-39 Up"      
[27] "Breast - Mammary Tissue Male 20-29 Up"       
[28] "Adipose - Subcutaneous Female 40-49 Up"      
[29] "Kidney - Cortex Male 40-49 Up"               
[30] "Adipose - Subcutaneous Female 70-79 Up"      
[31] "Breast - Mammary Tissue Female 70-79 Up"     
[32] "Cervix - Ectocervix Female 40-49 Up"

[1] "Adipose, Adrenal Gland, Breast, Cervix, Kidney, Liver"

In [ ]:
gtex_plier2_gtex_summary  %>% 
dplyr::filter(LV == "LV15")  %>% 
dplyr::pull(pathway)  %>%
  unique()

  lv15_df %>%
    slice_head(n = top_n) %>%
    pull(tissue) %>%
    sub(" -.*", "", .) %>%
    unique() %>%
    paste(collapse = "; ") %>%
    paste0("Top tissues: ", .)

[1] "Liver Male 60-69 Up, Liver Male 30-39 Up, Liver Female 60-69 Up, Liver Male 20-29 Up, Adrenal Gland Female 30-39 Up, Liver Male 40-49 Up, Kidney - Cortex Male 40-49 Up, Liver Male 70-79 Up, Adrenal Gland Male 70-79 Up, Liver Female 30-39 Up"

In [57]:
kegg_list$LV15@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Peroxisome"                                       
 [2] "Fatty acid degradation"                           
 [3] "Valine, leucine and isoleucine degradation"       
 [4] "Carbon metabolism"                                
 [5] "Biosynthesis of cofactors"                        
 [6] "Glycine, serine and threonine metabolism"         
 [7] "Fatty acid metabolism"                            
 [8] "Tryptophan metabolism"                            
 [9] "beta-Alanine metabolism"                          
[10] "PPAR signaling pathway"                           
[11] "Complement and coagulation cascades"              
[12] "One carbon pool by folate"                        
[13] "Alcoholic liver disease"                          
[14] "Pyruvate metabolism"                              
[15] "Propanoate metabolism"                            
[16] "Drug metabolism - cytochrome P450"                
[17] "Tyrosine metabolism"                              
[18] "Metabolism of xenobiotics by cytochrome P450"     
[19] "Butanoate metabolism"                             
[20] "Histidine metabolism"                             
[21] "Glyoxylate and dicarboxylate metabolism"          
[22] "Retinol metabolism"                               
[23] "Biosynthesis of unsaturated fatty acids"          
[24] "Arginine and proline metabolism"                  
[25] "Folate biosynthesis"                              
[26] "Arginine biosynthesis"                            
[27] "Alanine, aspartate and glutamate metabolism"      
[28] "Biosynthesis of amino acids"                      
[29] "Nicotinate and nicotinamide metabolism"           
[30] "Folate transport and metabolism"                  
[31] "Pantothenate and CoA biosynthesis"                
[32] "Cysteine and methionine metabolism"               
[33] "Chemical carcinogenesis - DNA adducts"            
[34] "Glycolysis / Gluconeogenesis"                     
[35] "Vitamin digestion and absorption"                 
[36] "Cholesterol metabolism"                           
[37] "Selenocompound metabolism"                        
[38] "Drug metabolism - other enzymes"                  
[39] "2-Oxocarboxylic acid metabolism"                  
[40] "Glutathione metabolism"                           
[41] "Ferroptosis"                                      
[42] "ABC transporters"                                 
[43] "Glycerolipid metabolism"                          
[44] "Citrate cycle (TCA cycle)"                        
[45] "Lysine degradation"                               
[46] "Pentose phosphate pathway"                        
[47] "Terpenoid backbone biosynthesis"                  
[48] "Fatty acid elongation"                            
[49] "Steroid biosynthesis"                             
[50] "Lipoic acid metabolism"                           
[51] "Biosynthesis of nucleotide sugars"                
[52] "Sulfur metabolism"                                
[53] "Primary bile acid biosynthesis"                   
[54] "Glycerophospholipid metabolism"                   
[55] "Insulin resistance"                               
[56] "AMPK signaling pathway"                           
[57] "Pyrimidine metabolism"                            
[58] "Fat digestion and absorption"                     
[59] "Antifolate resistance"                            
[60] "Mineral absorption"                               
[61] "Nucleotide metabolism"                            
[62] "Steroid hormone biosynthesis"                     
[63] "Chemical carcinogenesis - reactive oxygen species"
[64] "Proximal tubule bicarbonate reclamation"          
[65] "Adipocytokine signaling pathway"                  
[66] "Amino sugar and nucleotide sugar metabolism"      
[67] "Glucagon signaling pathway"

In [58]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV15")

LV,pathway
<chr>,<chr>
LV15,"Drug metabolism, Glycine, serine and threonine metabolism, Peroxisome, Retinol metabolism, Tyrosine metabolism"
